In [ ]:
# Napisz program serwera, który działając pod adresem 127.0.0.1 oraz na określonym porcie TCP będzie
# serwerem echa, który będzie odsyłał podłączającym się klientom odebrane od nich wiadomości.
import socket
import threading

def handle_client(client_socket, client_address):
    print(f"Połączono z {client_address}")
    with client_socket:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            print(f"Otrzymano od {client_address}: {data.decode('utf-8')}")
            client_socket.sendall(data)
    print(f"Rozłączono {client_address}")

def echo_server(host='127.0.0.1', port=12345):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((host, port))
        server_socket.listen(5)
        print(f"Serwer echa działa na {host}:{port}")

        while True:
            client_socket, client_address = server_socket.accept()
            client_thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
            client_thread.start()

if __name__ == "__main__":
    echo_server()

In [14]:
import socket
def test_echo_server(host='127.0.0.1', port=12345):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        client_socket.connect((host, port))
        message = "asfaf sf asfasf!"
        client_socket.sendall(message.encode('utf-8'))
        response = client_socket.recv(1024)
        print(f"respomnse: {response.decode('utf-8')}")


test_echo_server()

respomnse: asfaf sf asfasf!


In [ ]:
import socket
import threading
from datetime import datetime

LOG_FILE = "server_log.txt"

def log_event(message):
    with open(LOG_FILE, "a") as log_file:
        log_file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")

def handle_client(client_socket, client_address):
    log_event(f"New connection from {client_address[0]}:{client_address[1]}")
    print(f"Połączono z {client_address}")

    with client_socket:
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            message = data.decode('utf-8')
            print(f"Otrzymano od {client_address}: {message}")
            log_event(f"Received message from {client_address[0]}:{client_address[1]}: {message}")
            client_socket.sendall(data)
            log_event(f"Echoed message to {client_address[0]}:{client_address[1]}")
    
    log_event(f"Connection closed with {client_address[0]}:{client_address[1]}")
    print(f"Rozłączono {client_address}")

def echo_server(host='127.0.0.1', port=12345):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((host, port))
        server_socket.listen(5)
        print(f"Serwer echa działa na {host}:{port}")
        log_event(f"Server started on {host}:{port}")

        while True:
            client_socket, client_address = server_socket.accept()
            thread = threading.Thread(target=handle_client, args=(client_socket, client_address))
            thread.daemon = True  # wątek zakończy się, jeśli główny program się zamknie
            thread.start()

if __name__ == "__main__":
    echo_server()

In [18]:
import socket
def test_echo_server(host='127.0.0.1', port=12345):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        client_socket.connect((host, port))
        message = "adadadada sf asfasf!"
        client_socket.sendall(message.encode('utf-8'))
        response = client_socket.recv(1024)
        print(f"respomnse: {response.decode('utf-8')}")
test_echo_server()

respomnse: adadadada sf asfasf!


In [ ]:
import socket
import random
from datetime import datetime

HOST = '127.0.0.1'
PORT = 12345
LOG_FILE = "guess_server_log.txt"

def log_event(message):
    with open(LOG_FILE, "a") as log_file:
        log_file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {message}\n")

def guess_number_server(host=HOST, port=PORT):
    number_to_guess = random.randint(1, 100)
    print(f"[INFO] Serwer wystartował na {host}:{port}")
    print(f"[INFO] Wylosowana liczba: {number_to_guess}")
    log_event(f"Server started on {host}:{port} | Number to guess: {number_to_guess}")

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
        server_socket.bind((host, port))
        server_socket.listen(1)
        print(f"[INFO] Oczekiwanie na połączenie...")

        client_socket, client_address = server_socket.accept()
        log_event(f"New connection from {client_address[0]}:{client_address[1]}")
        with client_socket:
            print(f"[INFO] Połączono z {client_address}")
            client_socket.sendall(b"Witaj w grze 'Zgadnij liczbe'! Zgadnij liczbe od 1 do 100.\n")

            while True:
                data = client_socket.recv(1024).decode('utf-8').strip()
                if not data:
                    break

                log_event(f"Received from {client_address[0]}:{client_address[1]}: {data}")
                
                try:
                    guess = int(data)
                    if guess < number_to_guess:
                        response = "Za malo\n"
                    elif guess > number_to_guess:
                        response = "Za duzo\n"
                    else:
                        response = "Gratulacje! Zgadles liczbe!\n"
                        client_socket.sendall(response.encode('utf-8'))
                        log_event(f"Client guessed the number correctly: {guess}")
                        break
                except ValueError:
                    response = "Blad: Podaj poprawna liczbe calkowita.\n"

                client_socket.sendall(response.encode('utf-8'))

    print("[INFO] Zakonczono gre. Serwer wylosuje nowa liczbe przy ponownym uruchomieniu.")

if __name__ == "__main__":
    guess_number_server()

In [19]:
import socket

def guess_number_client(host='127.0.0.1', port=12345):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as sock:
        sock.connect((host, port))
        print(sock.recv(1024).decode('utf-8'))  # Powitanie

        while True:
            msg = input("Twoj strzal: ")
            sock.sendall(msg.encode('utf-8'))
            response = sock.recv(1024).decode('utf-8')
            print(f"Serwer: {response.strip()}")
            if "Gratulacje" in response:
                break

if __name__ == "__main__":
    guess_number_client()

Witaj w grze 'Zgadnij liczbe'! Zgadnij liczbe od 1 do 100.

Serwer: Za duzo
Serwer: Za malo
Serwer: Gratulacje! Zgadles liczbe!
